In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from impyute.imputation.cs import fast_knn

from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer 

from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

#models
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


C:\Users\bipin\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
file = open(r'labels.txt',  encoding='utf-8')
complete_label_list = [line.strip() for line in file.readlines()]

In [3]:
df_labeled= pd.read_csv('cleanedHungarian.csv')
df_labeled.columns
combined_labeled=pd.read_csv('combined_labeled')
combined_labeled =pd.concat([combined_labeled,df_labeled])
combined_labeled.to_csv('combined_labeled', index=False)

C:\Users\bipin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [4]:
X = df_labeled.drop(axis = 1, labels = "58 num: diagnosis of heart disease (angiographic disease status)")
X = X.drop(axis=1, labels= complete_label_list[58:68])
X=X.dropna(axis =1, thresh = 50)

Y_arteries = df_labeled[complete_label_list[58:68]] 
Y_arteries = Y_arteries.replace(to_replace = np.NAN, value = 0)

Y_target=df_labeled[complete_label_list[57]]
Y_target=Y_target.replace(2,1)
Y_target=Y_target.replace(3,1)
Y_target=Y_target.replace(4,1)
X.shape

(294, 29)

In [5]:
columns=[]
categorical_features=[]
numeric_features=[]
columns=X.columns
for ho in columns:
    if "1 =" in ho:
        categorical_features.append(ho)
    else:
        numeric_features.append(ho)
categorical_features.append('9 cp: chest pain type')

In [6]:
#X=fast_knn(X.values.astype(float), k=30) 
Y_arteries.columns

Index(['59 lmt', '60 ladprox', '61 laddist', '62 diag', '63 cxmain',
       '64 ramus', '65 om1', '66 om2', '67 rcaprox', '68 rcadist'],
      dtype='object')

In [7]:
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),('scaler', StandardScaler())])
categorical_transformer= Pipeline(steps=[('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [8]:
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),('cat', categorical_transformer, categorical_features)])

In [9]:
# #not in use
# def likelihood(top, bottom):
#     if bottom==0:
#         return 1
#     return (top+1)/bottom

# def recall(confusion_matrix):
#     column_sum = np.sum(confusion_matrix, axis=0)
#     num=0
#     conditional_probability=1
#     while num < len(confusion_matrix.T):
#         conditional_probability=conditional_probability+(confusion_matrix[num][num]+1)/confusion_matrix.sum()*likelihood(confusion_matrix[num][num],column_sum[num])
#         num=num+1
#     return conditional_probability
artery_recall =pd.DataFrame(columns = ["ANN", "SVM", "Tree", "NB"])

In [10]:
pipeline_SVM= Pipeline(steps=[('preprocessor', preprocessor),('pca1', PCA(n_components=5)),('clf_SVM',SVC(kernel = 'linear'))])

In [11]:
pipeline_ANN= Pipeline(steps=[('preprocessor', preprocessor),('pca1', PCA(n_components=5)),('ANN',MLPClassifier(solver='lbfgs', activation = 'logistic', hidden_layer_sizes = (500)))])

In [12]:
pipeline_DT= Pipeline(steps=[('preprocessor', preprocessor),('pca1', PCA(n_components=5)), ('clf_gini',DecisionTreeClassifier(criterion='gini'))])

In [13]:
pipeline_NB= Pipeline(steps=[('preprocessor', preprocessor),('pca1', PCA(n_components=5)),('clf_NB',GaussianNB())])

In [14]:
pipeline_Rf = Pipeline(steps=[('clf_RF_ext', RandomForestClassifier(max_depth=10000, random_state=5))])

In [15]:
pipelines = [pipeline_SVM, pipeline_ANN, pipeline_DT, pipeline_NB]
pipeline_dict= {0:"SVM",1:"ANN",2:"DT",3:"NB"}
df=pd.DataFrame()

In [16]:
arr=[]
for artery in Y_arteries.columns:
    #norm = 
    best_accuracy=0.0
    best_classifier=0
    best_pipeline=""
    accuracies=[]
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y_arteries[artery], test_size = 0.3,stratify = Y_arteries[artery])
    for pipe in pipelines:
        pipe.fit(X_train, Y_train)
     
    accuracies.append(artery)
    #checking accuracy
    #for i,model in enumerate(pipelines):
        #print("{} Test Accuracy: {}".format(pipeline_dict[i], model.score(X_test,Y_test)))
    for i,model in enumerate(pipelines):
        if model.score(X_test,Y_test)> best_accuracy:
            best_accuracy=model.score(X_test,Y_test)
            best_pipeline=model
            best_classifier =i
    arr.append(format(pipeline_dict[best_classifier]))
    
    predicted = pipelines[i].predict(X)
    df[artery]= predicted
print(arr)
df.columns

['NB', 'NB', 'SVM', 'SVM', 'SVM', 'SVM', 'SVM', 'SVM', 'SVM', 'SVM']


Index(['59 lmt', '60 ladprox', '61 laddist', '62 diag', '63 cxmain',
       '64 ramus', '65 om1', '66 om2', '67 rcaprox', '68 rcadist'],
      dtype='object')

In [17]:
#Experiment
#df.drop(columns=['60 ladprox', '67 rcaprox'])

#Experiment
#df.drop(columns=['60 ladprox', '67 rcaprox'])

In [18]:
#training for second tier
#pipeline_RF_PCA=Pipeline([('scalar5',StandardScaler()), ('pca5', PCA(n_components=3)),('clf_Rf',RandomForestClassifier(max_depth=10000, random_state=5))])

In [19]:
pipeline_SVM= Pipeline(steps=[('clf_SVM',SVC(kernel = 'linear'))])
pipeline_ANN= Pipeline(steps=[('ANN',MLPClassifier(solver='lbfgs', activation = 'logistic', hidden_layer_sizes = (500)))])
pipeline_DT= Pipeline(steps=[('clf_gini',DecisionTreeClassifier(criterion='gini'))])
pipeline_Rf = Pipeline(steps=[('clf_RF_ext', RandomForestClassifier(max_depth=10000, random_state=5))])

In [20]:
pipelines = [pipeline_SVM, pipeline_ANN, pipeline_DT, pipeline_Rf]
pipeline_dict= {0:"SVM",1:"ANN",2:"DT",3:"RF"}
df_mean= pd.DataFrame(columns=["Iteration","SVM","ANN","DT","RF"])
df_score=pd.read_csv(r'58_using_predicted_PCA')

In [21]:
#accuracies.append("Hungarian")
num=0
while num < 10:
    accuracies=[]
    X_train, X_test, Y_train, Y_test = train_test_split(df,Y_target, test_size = 0.2, stratify = Y_target)
    for pipe in pipelines:
        pipe.fit(X_train, Y_train)
    accuracies.append(num)
    #checking accuracy
    for i,model in enumerate(pipelines):
        print("{} Test Accuracy: {}".format(pipeline_dict[i], model.score(X_test,Y_test)))
        accuracies.append(model.score(X_test,Y_test))
    print("\n")
    print(accuracies)
    df_mean.loc[len(df_mean)]=accuracies
    num=num+1
print(len(df_score))

C:\Users\bipin\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


SVM Test Accuracy: 0.7457627118644068
ANN Test Accuracy: 0.7288135593220338
DT Test Accuracy: 0.7457627118644068
RF Test Accuracy: 0.7457627118644068


[0, 0.7457627118644068, 0.7288135593220338, 0.7457627118644068, 0.7457627118644068]
SVM Test Accuracy: 0.7796610169491526
ANN Test Accuracy: 0.7796610169491526
DT Test Accuracy: 0.7288135593220338
RF Test Accuracy: 0.7457627118644068


[1, 0.7796610169491526, 0.7796610169491526, 0.7288135593220338, 0.7457627118644068]
SVM Test Accuracy: 0.7288135593220338
ANN Test Accuracy: 0.7288135593220338
DT Test Accuracy: 0.7288135593220338
RF Test Accuracy: 0.7288135593220338


[2, 0.7288135593220338, 0.7288135593220338, 0.7288135593220338, 0.7288135593220338]
SVM Test Accuracy: 0.7796610169491526
ANN Test Accuracy: 0.7796610169491526
DT Test Accuracy: 0.7796610169491526
RF Test Accuracy: 0.7796610169491526


[3, 0.7796610169491526, 0.7796610169491526, 0.7796610169491526, 0.7796610169491526]
SVM Test Accuracy: 0.7966101694915254
ANN Test Accuracy:

In [22]:
print(df_mean)
results=np.array
results=df_mean.mean()

   Iteration       SVM       ANN        DT        RF
0        0.0  0.745763  0.728814  0.745763  0.745763
1        1.0  0.779661  0.779661  0.728814  0.745763
2        2.0  0.728814  0.728814  0.728814  0.728814
3        3.0  0.779661  0.779661  0.779661  0.779661
4        4.0  0.796610  0.813559  0.779661  0.762712
5        5.0  0.661017  0.661017  0.661017  0.677966
6        6.0  0.694915  0.694915  0.694915  0.694915
7        7.0  0.728814  0.728814  0.728814  0.728814
8        8.0  0.677966  0.677966  0.677966  0.677966
9        9.0  0.677966  0.677966  0.677966  0.677966


In [23]:
results.drop(labels='Iteration')

SVM    0.727119
ANN    0.727119
DT     0.720339
RF     0.722034
dtype: float64

In [24]:
df_score.loc[len(df_score)]=results
print(df_score)

       Dataset       SVM       ANN        DT        RF
0    Cleveland  0.768421  0.761404  0.754386  0.764912
1    Hungarian  0.737288  0.735593  0.740678  0.740678
2    LongBeach  0.750000  0.720000  0.732500  0.742500
3  Swizterland  0.920000  0.912000  0.908000  0.916000
4          NaN  0.727119  0.727119  0.720339  0.722034


In [25]:
#df_score.to_csv('58_using_predicted_PCA', index=False)